In [1]:
# библиотеки

import numpy as np  # для математических вычислений
import sympy as sp  # для символьного вычисления
from scipy.optimize import minimize_scalar  # для одномерной задачи минимизации (для alpha)

In [2]:
# глобальные переменные

x, y = sp.symbols('x y')
X = sp.Matrix([x, y])  # вектор из переменных
alpha = sp.symbols('alpha')
f = 1.5 * y ** 2 + x ** 2 + y - x + 3  # функция
grad = sp.Matrix([sp.diff(f, X_i) for X_i in X])  # градиент
start_point = sp.Matrix([0.4, -0.23])
eps = 0.01

In [3]:
def error_value(approximate_point, actual_point):
    """
    Функция для вычисления ошибки между реальным корнем, и корнем, полученным численно
    """
    delta_x = approximate_point - actual_point  # разность точек
    delta_x = delta_x[0] ** 2 + delta_x[1] ** 2  # расстояние между точками
    delta_func_val = f.subs(zip(X, approximate_point)) - f.subs(zip(X, actual_point))  # разность значений

    print("Разница точки: ", delta_x)
    print("Разница значения: ", delta_func_val)


def print_data(X_point, k):
    """
    Функция для вывода информации о конкретном шаге численного решения
    """
    print("Номер итерации: ", k)
    print("Point: ", list(X_point))
    print("Gradient: ", list(grad.subs(zip(X, X_point))))
    print()

# Метод сопряженных градиентов

In [4]:
def stopping_criteria_conjugate_gradient(h):
    """
    Функция, возвращающая true, если критерий остановки выполнен и нужно остановиться для метода сопряженных градиентов
    """
    if h is None:
        return False
    return np.dot(list(h), list(h)) < eps


def calc_new_point_conjugate_gradient(X_point, h):
    """
    Вычисление следуюшего шага методом сопряженных градиентов
    """
    h_next = -grad.subs(zip(X, X_point))  # антиградиент
    if h is not None:
        betta = np.dot(list(h_next), list(h_next)) / np.dot(list(h), list(h))  # считаем betta
        h_next += betta * h  # считаем новый h

    X_point = X_point + (h_next * alpha)  # считаем новую точку
    alpha_min = minimize_scalar(sp.lambdify([alpha], f.subs(zip(X, X_point)))).x  # минимизируем alpha
    X_point = X_point.subs(alpha, alpha_min)  # подставляем alpha
    return X_point, h_next


def conjugate_gradient():
    """
    Функция, для нахождения корня уравнения методом наискорейшего градинетного спуска
    """
    print("Метод сопряженных градиентов\n")

    k = 0
    h = None
    X_point = sp.Matrix(start_point)

    print_data(X_point, k)
    while not stopping_criteria_conjugate_gradient(h):
        X_point, h = calc_new_point_conjugate_gradient(X_point, h)  # вычисляем новую точку
        k += 1
        print_data(X_point, k)

    print("Всего итераций: ", k)
    print("Point: ", list(X_point))
    func_value = f.subs(zip(X, X_point))
    print("Function value: ", func_value)
    print()

    return X_point, func_value

# Наискорейший градиентный спуск

In [5]:
def stopping_criteria_steepest_descent_gradient(X_point):
    """
    Функция, возвращающая true, если критерий остановки выполнен и нужно остановиться для наискорейшего градиентного спуска
    """
    return np.max([np.abs(sp.diff(f, X_i).subs(zip(X, X_point))) for X_i in X]) < eps


def calc_new_point_steepest_descent_gradient(X_point):
    """
    Вычисление следуюшего шага методом наискорейшего градинетного спуска
    """
    X_point = X_point + (grad * sp.Matrix([alpha])).subs(zip(X, X_point))  # находим новую точку
    alpha_min = minimize_scalar(sp.lambdify([alpha], f.subs(zip(X, X_point)))).x  # минимизируем alpha
    X_point = X_point.subs(alpha, alpha_min)  # подставляем alpha
    return X_point


def steepest_descent_gradient():
    """
    Функция, для нахождения корня уравнения методом наискорейшего градинетного спуска
    """
    print("Метод наискорейшего градиентного спуска\n")

    k = 0
    X_point = sp.Matrix(start_point)

    print_data(X_point, k)
    while not stopping_criteria_steepest_descent_gradient(X_point):
        X_point = calc_new_point_steepest_descent_gradient(X_point)  # вычисляем новую точку
        k += 1  # считаем кол-во итераций
        print_data(X_point, k)

    print("Всего итераций: ", k)
    print("Point: ", list(X_point))
    func_value = f.subs(zip(X, X_point))
    print("Function value: ", f.subs(zip(X, X_point)))
    print()

    return X_point, func_value

In [6]:
# метод наискорейшего градиентного спуска

approximate_point = steepest_descent_gradient()[0]
actual_point = sp.Matrix([1 / 2, -1 / 3])  # точка полученная аналитически

error_value(approximate_point, actual_point)

Метод наискорейшего градиентного спуска

Номер итерации:  0
Point:  [0.400000000000000, -0.230000000000000]
Gradient:  [-0.200000000000000, 0.310000000000000]

Номер итерации:  1
Point:  [0.473907144422272, -0.344556073854522]
Gradient:  [-0.0521857111554551, -0.0336682215635666]

Номер итерации:  2
Point:  [0.496656901551118, -0.329878802115336]
Gradient:  [-0.00668619689776329, 0.0103635936539929]

Номер итерации:  3
Point:  [0.499127686867178, -0.333708515091700]
Gradient:  [-0.00174462626564453, -0.00112554527510067]

Всего итераций:  3
Point:  [0.499127686867178, -0.333708515091700]
Function value:  2.58333430540556

Разница точки:  9.01691553505455e-7
Разница значения:  9.72072229821208e-7


In [7]:
# метод сопряженных направлений

approximate_point = conjugate_gradient()[0]
actual_point = sp.Matrix([1 / 2, -1 / 3])  # точка полученная аналитически

error_value(approximate_point, actual_point)

Метод сопряженных градиентов

Номер итерации:  0
Point:  [0.400000000000000, -0.230000000000000]
Gradient:  [-0.200000000000000, 0.310000000000000]

Номер итерации:  1
Point:  [0.473907148769748, -0.344556080593110]
Gradient:  [-0.0521857024605032, -0.0336682417793299]

Номер итерации:  2
Point:  [0.499999999878337, -0.333333330495203]
Gradient:  [-2.43325248838744e-10, 8.51439052595993e-9]

Всего итераций:  2
Point:  [0.499999999878337, -0.333333330495203]
Function value:  2.58333333333333

Разница точки:  8.06978468624240e-18
Разница значения:  0
